Code is a sequence of instructions.  
Instructions pop values from operand stack, and then push results to operand stack. For now, can push at most one result.  
Some sequence also have static immediate arguments, usually indices or type annotations.  

Some instructions are structed, and contains nested block of instructions. Used for if/else, loops.

# Numeric instructions

Basic operations over numeric values of some types.

- constants: eg i32.const 2: push constant value on stack
- unary operations: clz (count leading zeros), ctz (count trailing zeros), popcnt (nb bits set so 1), abs, neg, etc
- binary operations: add, sub, mul, 
- tests operations: consume one operand and push one boolean i32 result: eqz (== 0)
- compare operations: consume 2 operands, compare and push one boolean i32 result: eq, ne, lt, gt, etc
- conversions operations: extend or truncate between i32 and i64, or convert between f32, f64, and between int and float.

# Parametric instructions

Work on any value type:
    
- drop: consume and ignore one operand
- select: select one of it's first 2 operands based on if the third is 0 or not.

# Variable instructions

access local and global variables:
- get value to stack: local.get, global.get
- save value from stack: local.set, global.set
- save but doesn't pop stack: local.tee, global.tee

```c
int fact(int x)
{
  return x <= 1 ? 1 : x * fact(x-1);
}

int add(int a, int b)
{
  return a + b;
}

int my_pow(int x, int n)
{
  if (n == 0)
    return 1;
  else if (n == 1)
    return x;
  else if (n == 2)
    return x*x;
  else if (n % 2 == 0)
    return my_pow(x*x, n/2);
  else
    return my_pow(x*x, n/2) * x;
}

int my_abs(int x)
{
  return x <= 0 ? -x : x;
}

int is_null(int x)
{
  return x == 0;
}
```

```
math.wasm:	file format wasm 0x1

Code Disassembly:

0000a3 func[0] <__post_instantiate>:
 0000a4: 01                         | nop
 0000a5: 0b                         | end
0000a7 func[1] <fact>:
 0000a8: 20 00                      | local.get 0
 0000aa: 41 02                      | i32.const 2
 0000ac: 4e                         | i32.ge_s
 0000ad: 04 7f                      | if i32
 0000af: 20 00                      |   local.get 0
 0000b1: 41 7f                      |   i32.const 4294967295
 0000b3: 6a                         |   i32.add
 0000b4: 10 01                      |   call 1 <fact>
 0000b6: 20 00                      |   local.get 0
 0000b8: 6c                         |   i32.mul
 0000b9: 05                         | else
 0000ba: 41 01                      |   i32.const 1
 0000bc: 0b                         | end
 0000bd: 0b                         | end
0000bf func[2] <add>:
 0000c0: 20 00                      | local.get 0
 0000c2: 20 01                      | local.get 1
 0000c4: 6a                         | i32.add
 0000c5: 0b                         | end
0000c7 func[3] <my_pow>:
 0000c8: 01 7f                      | local[0] type=i32
 0000ca: 02 40                      | block
 0000cc: 02 40                      |   block
 0000ce: 20 01                      |     local.get 1
 0000d0: 41 02                      |     i32.const 2
 0000d2: 4d                         |     i32.le_u
 0000d3: 04 40                      |     if
 0000d5: 41 01                      |       i32.const 1
 0000d7: 21 02                      |       local.set 2
 0000d9: 02 40                      |       block
 0000db: 20 01                      |         local.get 1
 0000dd: 41 01                      |         i32.const 1
 0000df: 6b                         |         i32.sub
 0000e0: 0e 02 02 00 03             |         br_table 2 0 3
 0000e5: 0b                         |       end
 0000e6: 20 00                      |       local.get 0
 0000e8: 20 00                      |       local.get 0
 0000ea: 6c                         |       i32.mul
 0000eb: 0f                         |       return
 0000ec: 0b                         |     end
 0000ed: 20 00                      |     local.get 0
 0000ef: 20 00                      |     local.get 0
 0000f1: 6c                         |     i32.mul
 0000f2: 20 01                      |     local.get 1
 0000f4: 41 02                      |     i32.const 2
 0000f6: 6d                         |     i32.div_s
 0000f7: 10 03                      |     call 3 <my_pow>
 0000f9: 20 00                      |     local.get 0
 0000fb: 41 01                      |     i32.const 1
 0000fd: 20 01                      |     local.get 1
 0000ff: 41 01                      |     i32.const 1
 000101: 71                         |     i32.and
 000102: 1b                         |     select
 000103: 6c                         |     i32.mul
 000104: 0f                         |     return
 000105: 0b                         |   end
 000106: 20 00                      |   local.get 0
 000108: 21 02                      |   local.set 2
 00010a: 0b                         | end
 00010b: 20 02                      | local.get 2
 00010d: 0b                         | end
00010f func[4] <my_abs>:
 000110: 01 7f                      | local[0] type=i32
 000112: 20 00                      | local.get 0
 000114: 20 00                      | local.get 0
 000116: 41 1f                      | i32.const 31
 000118: 75                         | i32.shr_s
 000119: 22 01                      | local.tee 1
 00011b: 6a                         | i32.add
 00011c: 20 01                      | local.get 1
 00011e: 73                         | i32.xor
 00011f: 0b                         | end
000121 func[5] <is_null>:
 000122: 20 00                      | local.get 0
 000124: 45                         | i32.eqz
 000125: 0b                         | end
```

# Memory Instructions

Read/write data of linear memory

- load: load from memory at a specific offset and alignment
- store: load to memory at a specific offset and alignment

a trap occurs if trying to read/write outside of memory range.  
memory adress stored as i32.

- memory.size: returns current memory size
- memory.grow: expand memory size by an offset
both operations operate on unit of page size

```c
int my_data[18];


int get_it()
{
  return my_data[6];
}

void set_it(int x)
{
  my_data[6] = x;
}

void add_it(int x)
{
  my_data[6] += x;
}
```

```
mem.wasm:	file format wasm 0x1

Code Disassembly:

0000dc func[1]:
 0000dd: 10 02                      | call 2 <__wasm_apply_relocs>
 0000df: 0b                         | end
0000e1 func[2] <__wasm_apply_relocs>:
 0000e2: 01                         | nop
 0000e3: 0b                         | end
0000e5 func[3] <get_it>:
 0000e6: 23 01                      | global.get 1
 0000e8: 28 02 18                   | i32.load 2 24
 0000eb: 0b                         | end
0000ed func[4] <set_it>:
 0000ee: 23 01                      | global.get 1
 0000f0: 20 00                      | local.get 0
 0000f2: 36 02 18                   | i32.store 2 24
 0000f5: 0b                         | end
0000f7 func[5] <add_it>:
 0000f8: 01 7f                      | local[0] type=i32
 0000fa: 23 01                      | global.get 1
 0000fc: 22 01                      | local.tee 1
 0000fe: 20 01                      | local.get 1
 000100: 28 02 18                   | i32.load 2 24
 000103: 20 00                      | local.get 0
 000105: 6a                         | i32.add
 000106: 36 02 18                   | i32.store 2 24
 000109: 0b                         | end
00010b func[6] <__post_instantiate>:
 00010c: 10 07                      | call 7
 00010e: 10 01                      | call 1
 000110: 0b                         | end
000112 func[7]:
 000113: 10 00                      | call 0 <env.g$my_data>
 000115: 24 01                      | global.set 1
 000117: 0b                         | end
```

# Control instructions

Control instructions affect the control flow

- nop
- unreachable: trap
- block: define a structued instruction
- if else: structured instruction
- loop: structued instruction
- br: branch to a label
- br_if: conditionaly branch to a label
- br_table: branch given a table of labels and an operand index in this table
- return: shortcut to jump to the outermost block
- call: invoke another functions, consuming arguments from operands stack, and returning result on top of stack.
- call_indirect: call a function in table, through an operand index.

```c
int foo(int x)
{
  int y = x * 2;
  if (x > 10)
  {
    int z = x * 3;
    if (x > z)
      x += 5;
    else if (x < 3)
      x *= 100;
  }
  else {
    x *= y + 67;
  }

  return x + y;
}
```

```
00008d func[2] <foo>:
 00008e: 01 7f                      | local[0] type=i32
 000090: 20 00                      | local.get 0
 000092: 41 01                      | i32.const 1
 000094: 74                         | i32.shl
 000095: 21 01                      | local.set 1
 000097: 20 00                      | local.get 0
 000099: 41 0b                      | i32.const 11
 00009b: 4e                         | i32.ge_s
 00009c: 04 40                      | if
 00009e: 20 00                      |   local.get 0
 0000a0: 41 03                      |   i32.const 3
 0000a2: 6c                         |   i32.mul
 0000a3: 20 00                      |   local.get 0
 0000a5: 48                         |   i32.lt_s
 0000a6: 04 40                      |   if
 0000a8: 20 00                      |     local.get 0
 0000aa: 41 05                      |     i32.const 5
 0000ac: 6a                         |     i32.add
 0000ad: 20 01                      |     local.get 1
 0000af: 6a                         |     i32.add
 0000b0: 0f                         |     return
 0000b1: 0b                         |   end
 0000b2: 20 00                      |   local.get 0
 0000b4: 41 e4 00                   |   i32.const 100
 0000b7: 6c                         |   i32.mul
 0000b8: 20 00                      |   local.get 0
 0000ba: 20 00                      |   local.get 0
 0000bc: 41 03                      |   i32.const 3
 0000be: 48                         |   i32.lt_s
 0000bf: 1b                         |   select
 0000c0: 20 01                      |   local.get 1
 0000c2: 6a                         |   i32.add
 0000c3: 0f                         |   return
 0000c4: 0b                         | end
 0000c5: 20 01                      | local.get 1
 0000c7: 41 c3 00                   | i32.const 67
 0000ca: 6a                         | i32.add
 0000cb: 20 00                      | local.get 0
 0000cd: 6c                         | i32.mul
 0000ce: 20 01                      | local.get 1
 0000d0: 6a                         | i32.add
 0000d1: 0b                         | end
```

```c
int bar(int x, int y)
{
  if (x < 0 || y < 0)
    return x + y;
  else if (x > y)
    return x;
  else if (x < y)
    return y;
  else
    return 100;
}
```

```
0000d3 func[3] <bar>:
 0000d4: 20 00                      | local.get 0
 0000d6: 20 01                      | local.get 1
 0000d8: 72                         | i32.or
 0000d9: 41 7f                      | i32.const 4294967295
 0000db: 4c                         | i32.le_s
 0000dc: 04 40                      | if
 0000de: 20 00                      |   local.get 0
 0000e0: 20 01                      |   local.get 1
 0000e2: 6a                         |   i32.add
 0000e3: 0f                         |   return
 0000e4: 0b                         | end
 0000e5: 20 01                      | local.get 1
 0000e7: 41 e4 00                   | i32.const 100
 0000ea: 20 00                      | local.get 0
 0000ec: 20 01                      | local.get 1
 0000ee: 48                         | i32.lt_s
 0000ef: 1b                         | select
 0000f0: 20 00                      | local.get 0
 0000f2: 20 00                      | local.get 0
 0000f4: 20 01                      | local.get 1
 0000f6: 4c                         | i32.le_s
 0000f7: 1b                         | select
 0000f8: 0b                         | end
```

```c
void foo_n(int n)
{
  for (int i = 0; i < n; ++i)
    foo();
}
```

```
0000b6 func[3] <foo_n>:
 0000b7: 01 7f                      | local[0] type=i32
 0000b9: 41 00                      | i32.const 0
 0000bb: 21 01                      | local.set 1
 0000bd: 20 00                      | local.get 0
 0000bf: 41 00                      | i32.const 0
 0000c1: 4a                         | i32.gt_s
 0000c2: 04 40                      | if
 0000c4: 03 40                      |   loop
 0000c6: 10 00                      |     call 0 <env.foo>
 0000c8: 20 01                      |     local.get 1
 0000ca: 41 01                      |     i32.const 1
 0000cc: 6a                         |     i32.add
 0000cd: 22 01                      |     local.tee 1
 0000cf: 20 00                      |     local.get 0
 0000d1: 47                         |     i32.ne
 0000d2: 0d 00                      |     br_if 0
 0000d4: 0b                         |   end
 0000d5: 0b                         | end
 0000d6: 0b                         | end
```

```c
int fact(int x)
{
  int res = 1;
  while (x > 1)
  {
    res *= x;
    x -= 1;
  }
  return res;
}
```

```
0000d8 func[4] <fact>:
 0000d9: 02 7f                      | local[0..1] type=i32
 0000db: 41 01                      | i32.const 1
 0000dd: 21 01                      | local.set 1
 0000df: 20 00                      | local.get 0
 0000e1: 41 02                      | i32.const 2
 0000e3: 4e                         | i32.ge_s
 0000e4: 04 40                      | if
 0000e6: 03 40                      |   loop
 0000e8: 20 00                      |     local.get 0
 0000ea: 20 01                      |     local.get 1
 0000ec: 6c                         |     i32.mul
 0000ed: 21 01                      |     local.set 1
 0000ef: 20 00                      |     local.get 0
 0000f1: 41 02                      |     i32.const 2
 0000f3: 4a                         |     i32.gt_s
 0000f4: 21 02                      |     local.set 2
 0000f6: 20 00                      |     local.get 0
 0000f8: 41 7f                      |     i32.const 4294967295
 0000fa: 6a                         |     i32.add
 0000fb: 21 00                      |     local.set 0
 0000fd: 20 02                      |     local.get 2
 0000ff: 0d 00                      |     br_if 0
 000101: 0b                         |   end
 000102: 0b                         | end
 000103: 20 01                      | local.get 1
 000105: 0b                         | end
```


```c
int el_break(int x)
{

  int res = 0;

  for (int i = x / 3; i < x; ++i)
  {
    if (i % 4 == 0)
      break;
    ++res;
  }

  return res;
}
```

```
000108 func[5] <el_break>:
 000109: 02 7f                      | local[0..1] type=i32
 00010b: 41 00                      | i32.const 0
 00010d: 21 01                      | local.set 1
 00010f: 02 40                      | block
 000111: 20 00                      |   local.get 0
 000113: 41 03                      |   i32.const 3
 000115: 6d                         |   i32.div_s
 000116: 22 02                      |   local.tee 2
 000118: 20 00                      |   local.get 0
 00011a: 4e                         |   i32.ge_s
 00011b: 0d 00                      |   br_if 0
 00011d: 20 02                      |   local.get 2
 00011f: 41 03                      |   i32.const 3
 000121: 71                         |   i32.and
 000122: 45                         |   i32.eqz
 000123: 0d 00                      |   br_if 0
 000125: 20 02                      |   local.get 2
 000127: 41 7f                      |   i32.const 4294967295
 000129: 73                         |   i32.xor
 00012a: 22 01                      |   local.tee 1
 00012c: 20 00                      |   local.get 0
 00012e: 6a                         |   i32.add
 00012f: 22 00                      |   local.tee 0
 000131: 20 01                      |   local.get 1
 000133: 41 03                      |   i32.const 3
 000135: 71                         |   i32.and
 000136: 22 01                      |   local.tee 1
 000138: 20 00                      |   local.get 0
 00013a: 20 01                      |   local.get 1
 00013c: 49                         |   i32.lt_u
 00013d: 1b                         |   select
 00013e: 41 01                      |   i32.const 1
 000140: 6a                         |   i32.add
 000141: 21 01                      |   local.set 1
 000143: 0b                         | end
 000144: 20 01                      | local.get 1
 000146: 0b                         | end
```

```c
int el_continue(int x)
{

  int res = 0;

  for (int i = x / 3; i < x; ++i)
  {
    if (i % 4 == 0)
      continue;
    ++res;
  }

  return res;
}
```

```
000148 func[6] <el_continue>:
 000149: 02 7f                      | local[0..1] type=i32
 00014b: 41 00                      | i32.const 0
 00014d: 21 01                      | local.set 1
 00014f: 20 00                      | local.get 0
 000151: 41 03                      | i32.const 3
 000153: 6d                         | i32.div_s
 000154: 22 02                      | local.tee 2
 000156: 20 00                      | local.get 0
 000158: 48                         | i32.lt_s
 000159: 04 40                      | if
 00015b: 03 40                      |   loop
 00015d: 20 01                      |     local.get 1
 00015f: 20 02                      |     local.get 2
 000161: 41 03                      |     i32.const 3
 000163: 71                         |     i32.and
 000164: 41 00                      |     i32.const 0
 000166: 47                         |     i32.ne
 000167: 6a                         |     i32.add
 000168: 21 01                      |     local.set 1
 00016a: 20 02                      |     local.get 2
 00016c: 41 01                      |     i32.const 1
 00016e: 6a                         |     i32.add
 00016f: 22 02                      |     local.tee 2
 000171: 20 00                      |     local.get 0
 000173: 47                         |     i32.ne
 000174: 0d 00                      |     br_if 0
 000176: 0b                         |   end
 000177: 0b                         | end
 000178: 20 01                      | local.get 1
 00017a: 0b                         | end
```